Code for processing the data that I have, aligning labels for decoding
Should import with py-scipystack
For working with Sherlock's interface, need to import modules. Need to have specific module loading for it to work otherwise go to the input.sh and run those commands in this notebook. 
(here is the sequence that works: py-scipystack math py-autograd py-pytorch cuda praat )

What features do we want?

We have:
1. phonemes per time
2. silence
3. F0 pitch per time (individually on Praat)
4. dB per time (individually audio on Praat with line separation)

TODO: (see more planning doc for more scripts)
1. Volume (db); this is described as "intensity" on praat
2. pitch characteristics (spectrogram?) (can get pitch Hz from the github script on praat: https://github.com/lennes/pitch-distributions/blob/master/collectPitchSamplesFromCorpus.praat)
   1. idea here: may be able to use strongest power frequencies or extract from Praat (corroborate from Praat)
   2. corroborate with MFA phonemes that are vocal

Decoder characteristics:
1. silence vs sound
2. speech envelope (volume)
3. maybe discretize pitch first into a few categories
   -for pitch, probably need to normalize? and do relative pitch from average or something

Create jupyter instance
check dependencies
figure out how to export Praat alignments and textgrids
parse those into bins
align audios with the cues that we have
align labels with cues that we have

other question: do we need to align at all? can bypass and use bin time of 20ms to align. 


Task audio, mic audio

Use 'trial_start_nsp_analog_time', 'trial_end_nsp_analog_time'
To align with nap time

Check with last trial end in ns5

Audio at 30KHz

Can play audio back from ns5

20ms binsize

maybe also get the neural PSTHs around word onset?

#delay time start
#check if there is a delay with the ns5 data (use blackrock documentation)
#load ns5 data--> one isneural, another is analog, other is digital
#this has audio stream. check to make sure audio stream is aligned with what we expect with the 20ms bin timing

In [2]:
#assume that labels are ready in 20ms time intervals
#assume we know alignment thru redis clock 

testin2g2


In [2]:
import math

In [1]:
import numpy as np

In [2]:
import scipy.io as sio

ts_filepath = '/home/groups/henderj/rzwang/exp_data/prosody/t12-02-20-2025/20250220_082208_(1).mat'
# 5. Load redis MATLAB file with nsp timestamp information.
mat_data = sio.loadmat(ts_filepath)

In [3]:
print(mat_data.keys())

['block_number', 'block_start_time', 'all_candidate_sentence_acoustic_scores', 'graph_name', 'confirmation_epoch_start_redis_time', 'trial_start_nsp_analog_time', 'trial_end_nsp_analog_time', 'delay_duration_ms', 'all_candidate_sentence_oldlm_scores', 'cue3_start_nsp_analog_time', 'decoder_signal', 'all_candidate_sentence_newlm_scores', 'binned_neural_spike_band_power', 'session_description', 'confirmation_epoch_start_nsp_analog_time', 'binned_neural_redis_clock', 'binned_neural_threshold_crossings', 'task_type', 'participant', 'trial_start_redis_time', 'corrected_with_candidate', 'microphone_nsp_time', 'correct_status', 'inter_trial_duration_ms', 'trial_end_nsp_neural_time', 'cue', 'all_candidate_sentences', 'block_description', 'ngram_decoder_partial_output', '__version__', 'word_count_mismatch', 'norm_channel_stds', 'trial_accuracy_confirmation', 'binned_neural_nsp_timestamp', 'cue3', 'cue2_start_nsp_neural_time', 'trial_end_redis_time', 'cue3_start_redis_time', 'cue2_start_redis_ti

In [4]:
#mat_data probing. Let's go on redis clock, rather than nsp time (since bins are currently in redis time)
#bins should be 20ms bins
print(mat_data['binned_neural_redis_clock']) #starts at 1740068451564, ends 1740068502478. Probably the end of bins? or the start of bins
#todo: check whether bin times are start or end boundaries
print(mat_data['binned_neural_redis_clock'].shape) #1x2547 long
print(mat_data['block_start_time']) #1740068439
print(mat_data['binned_neural_spike_band_power'].shape) #array of floats, 2547x256 (time x array)
print(mat_data['binned_neural_threshold_crossings'].shape) #binary array, 2547x256 (time x array)
print(mat_data['task_type'])
print(mat_data['cue']) #array of cue lines (audio file, corresponding transcript, number of words. can use this to match trials with labels/audio files, potentially)
#side note: I also have block .txt files that have this information
print(mat_data['binned_neural_nsp_timestamp'])
print(mat_data['go_cue_nsp_neural_time'])
print(mat_data['go_cue_redis_time']) #go cues for each trial, on the redis clock
print(mat_data['trial_start_redis_time']) #trial start times on the redis clock
print(mat_data['trial_end_redis_time']) #trial end times on the redis clock


#I think it's the binned_neural_redis_clock

[[1740068451564 1740068451584 1740068451604 ... 1740068502438
  1740068502458 1740068502478]]
(1, 2547)
[[1740068439]]
(2547, 256)
(2547, 256)
[]
[u"scaled_100wpm_POD0000000240_S0000228.wav;AND IF IT'S MICHAEL;4;1.650000000000091;podcast;2.42;1.82;1.45                  "
 u'scaled_100wpm_POD0000000320_S0000205.wav;I CAN PLAY THE MOVIE;5;1.5399999999999636;podcast;3.25;2.44;1.95                '
 u"scaled_100wpm_POD0000000416_S0000055.wav;WHO ARE COGNIZANT OF WHAT'S HAPPENING;6;2.7299999999999613;podcast;2.2;1.65;1.32"
 u'scaled_100wpm_AUD0000001043_S0000823.wav;NONE FOR ME;3;1.2300000000000182;audiobook;2.44;1.83;1.46                       '
 u'scaled_100wpm_POD0000000352_S0000314.wav;MONDAY BEGAN LIKE A NORMAL DAY;6;1.8199999999999363;podcast;3.3;2.47;1.98       ']
[[1.72667492e+18 1.72667492e+18 1.72667492e+18 ... 1.72667497e+18
  1.72667497e+18 1.72667497e+18]]
[[1.72667492e+18 1.72667494e+18 1.72667495e+18 1.72667495e+18
  1.72667496e+18]]
[[1.74006846e+12 1.74006847e+12 1.74006848

In [5]:
block_start = mat_data['block_start_time'][0][0]  # 1740068439

bin_times = mat_data['binned_neural_redis_clock'][0]  # 1x2547 array

# Bins appear to be 20ms (difference between consecutive times is 20)
# Trial-specific timing

trial_starts = mat_data['trial_start_redis_time'][0]
trial_ends = mat_data['trial_end_redis_time'][0]
go_cues = mat_data['go_cue_redis_time'][0]
cues = mat_data['cue']  # Contains audio filename, transcript, word count, etc.

In [ ]:
def mat_to_npy(mat_file_path, output_dir):
    mat_data = sio.loadmat(mat_file_path)
    np.save(f"{output_dir}/spike_band_power.npy", mat_data['binned_neural_spike_band_power'])
    np.save(f"{output_dir}/threshold_crossings.npy",mat_data['binned_neural_threshold_crossings'])
    np.save(f"{output_dir}/bin_times.npy", mat_data['binned_neural_redis_clock'][0])
    np.save(f"{output_dir}/go_times.npy", mat_data['go_cue_redis_time'][0])
    np.save(f"{output_dir}/trial_starts.npy", mat_data['trial_start_redis_time'][0])
    np.save(f"{output_dir}/trial_ends.npy", mat_data['trial_start_redis_time'][0])